In [5]:
from pulp import *

proces_time = {'A1':{'TTC':6,'TTL':2,'C':6,'P': 4,'MM':5},
               'A2':{'TTC':10,'TTL':8,'C':4,'P':2,'MM':3},
               'A3':{'TTC':4,'TTL':3,'C':5,'P':7,'MM':4}}

# 
M = len(proces_time.keys()) # M = 3
N = len(proces_time['A1'].keys()) # N = 5

machine_names = [key for key in proces_time.keys()] # ['A1', 'A2', 'A3']
job_names = [key for key in proces_time['A1'].keys()] # ['TTC', 'TTL', 'C', 'P', 'MM']

# Ranges where indexes are defined
jobs = range(1,N+1)
machines = range(1,M+1)

# convert data to dictionary {(i,k): value}   (i=job, k=machine)
p = {(i,k): proces_time[machine_names[k-1]][job_names[i-1]] for i in jobs for k in machines}

# Create the model
prob = LpProblem("flowshop", LpMinimize)

# Variables
x = LpVariable.dicts("x", ((i,k) for i in range(N+1) for k in range(N+1)), cat='Binary')
s = LpVariable.dicts("s", ((k,j) for k in range(N+1) for j in range(M+1)), lowBound=0)
makespan = LpVariable("mksp", lowBound=0)

# Constraints
# One job is scheduled only once
for k in jobs: 
    prob += lpSum(x[(i,k)] for i in jobs) == 1

# Each possible order is used only once
for i in jobs: 
    prob += lpSum(x[(i,k)] for k in jobs) == 1

# Initial conditions
for j in machines:
    prob += s[0,j] == 0  # Dummy job 0 finishes at time 0

for k in jobs:
    prob += s[k,0] == 0  # Dummy machine 0 has zero processing time

# Machine timing constraints (take into acount the maximum betwen the previous machine of the same job and the next machine of the previous job)
for j in range(1, M+1):
    for k in jobs:
        prob += s[k,j] >= s[k,j-1] + lpSum(x[(i,k)] * p[(i,j)] for i in jobs)
        prob += s[k,j] >= s[k-1,j] + lpSum(x[(i,k)] * p[(i,j)] for i in jobs)
# Define makespan
prob += makespan == s[N,M]
# Objective: Minimize makespan
prob += makespan

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=True))

# Store solution in a dictionary sch_dict = {(job, machine): start_time} and order_dict = {order: job} = {1: jobname, 2: jobname, ...}
sch_dict = {}
order_dict = {}
x_names_ranges = {v: k for k, v in x.items()}
s_names_ranges = {v: k for k, v in s.items()}

In [6]:
makespan.varValue

35.0

In [7]:
makespan.name

'mksp'

In [8]:
makespan.value()

35.0

In [9]:
x.items()

dict_items([((0, 0), x_(0,_0)), ((0, 1), x_(0,_1)), ((0, 2), x_(0,_2)), ((0, 3), x_(0,_3)), ((0, 4), x_(0,_4)), ((0, 5), x_(0,_5)), ((1, 0), x_(1,_0)), ((1, 1), x_(1,_1)), ((1, 2), x_(1,_2)), ((1, 3), x_(1,_3)), ((1, 4), x_(1,_4)), ((1, 5), x_(1,_5)), ((2, 0), x_(2,_0)), ((2, 1), x_(2,_1)), ((2, 2), x_(2,_2)), ((2, 3), x_(2,_3)), ((2, 4), x_(2,_4)), ((2, 5), x_(2,_5)), ((3, 0), x_(3,_0)), ((3, 1), x_(3,_1)), ((3, 2), x_(3,_2)), ((3, 3), x_(3,_3)), ((3, 4), x_(3,_4)), ((3, 5), x_(3,_5)), ((4, 0), x_(4,_0)), ((4, 1), x_(4,_1)), ((4, 2), x_(4,_2)), ((4, 3), x_(4,_3)), ((4, 4), x_(4,_4)), ((4, 5), x_(4,_5)), ((5, 0), x_(5,_0)), ((5, 1), x_(5,_1)), ((5, 2), x_(5,_2)), ((5, 3), x_(5,_3)), ((5, 4), x_(5,_4)), ((5, 5), x_(5,_5))])

In [11]:
x_names = x.keys()

print('\nPRINTING keys of LpVariable.dicts (keys of the dict x)\n')
print(x_names)
print('\nPRINTING name and value of each x (each value of the dict)\n')
for one_x in x_names:
    var = x[one_x]
    if var.varValue is not None and var.varValue > 0:
        print(var.name,'=', var.varValue)
        # you can also use var.value() instead of var.varValue
        print(var.name,'=', var.value())  


PRINTING keys of LpVariable.dicts (keys of the dict x)

dict_keys([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)])

PRINTING name and value of each x (each value of the dict)

x_(1,_3) = 1.0
x_(1,_3) = 1.0
x_(2,_1) = 1.0
x_(2,_1) = 1.0
x_(3,_4) = 1.0
x_(3,_4) = 1.0
x_(4,_2) = 1.0
x_(4,_2) = 1.0
x_(5,_5) = 1.0
x_(5,_5) = 1.0
